- 2121221009 Nurgül ŞAHİN
- 2221221024 Öykü Azra YILMAZ
- 2221221046 İrem AYVAZ

**Problem definition :** TSP (Traveling Salesman Problem)
- Amaç: Tüm şehirleri bir kez ziyaret edip başlangıç noktasına dönecek en kısa rotayı bulmak
- Girdi: Şehirler arası mesafe matrisi
- Çıktı: Optimal/optimal-yakın rota ve toplam mesafe

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from pathlib import Path
import time
import traceback
import random
from sklearn.model_selection import ParameterGrid

random.seed(42)
np.random.seed(42)

### Gerekli Sınıflar

- **CITY :**  Şehri temsil eder.

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"City({self.x:.1f},{self.y:.1f})"

    def __str__(self):
        return f"({self.x:.1f}, {self.y:.1f})"
    
    def print_info(self):
        print(f"City Coordinates: {self.x}, {self.y}")

- **INDIVIDUAL :** Bir çözümü(rotayı) temsil eder.

In [ ]:
class Individual:
    def __init__(self, tour_order=None):
        self.tour_order = tour_order if tour_order else [] # şehir listesinin bir permütasyonu
        self.fitness = None # az tur uzunluğu, iyi fitness
        self.distance_value = None # toplam mesafe
    
    def calculate_distance(self, c1, c2): # şehirler arası uzaklık
        return math.hypot(c1.x - c2.x, c1.y - c2.y)
    
    def calculate_fitness(self, distance_matrix=None): # toplam rota mesafesi (toplam alınan yol)
        if self.fitness is not None: # fitness'ı zaten hesaplanmış
            return self.fitness

        if not self.tour_order or len(self.tour_order) == 0: # boş tour kontrolü
            self.fitness = 0
            self.distance_value = float('inf')
            return self.fitness
            
        if distance_matrix is None:  
            raise ValueError("Distance matrix gerekli! GA doğru initialize edilmedi.")
            
        total_distance = 0
        num_cities = len(self.tour_order)

        for i in range(num_cities):
            from_city = self.tour_order[i] # Başlangıçtan son şehre
            to_city = self.tour_order[(i + 1) % num_cities]  # Son şehirden başlangıca
            total_distance += distance_matrix[from_city][to_city]

        
        self.distance_value = total_distance
        self.fitness = 1 / (total_distance + 1e-10)  # Küçük mesafe = yüksek fitness
        return self.fitness

    def get_total_distance(self): # Toplam mesafeyi döndür
        return self.distance_value if self.distance_value is not None else float('inf')

    def reset_fitness(self): # Fitness değerlerini sıfırla (mutation/crossover sonrası)
        self.fitness = None
        self.distance_value = None

- **GENETIC ALGORITHM**

In [ ]:
class GA:
    def __init__(self, cities, pop_size=100, termination=1000, 
                 crossover_rate=0.8, mutation_rate=0.02, elitism_rate=0.1, tournament_size=5):
        self.cities = cities # şehir listesi
        self.pop_size = pop_size # population size
        self.population = []
        self.termination = termination # Kaç nesil çalışacak?
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elitism_count = int(pop_size * elitism_rate)
        self.num_cities = len(cities)
        self.tournament_size = tournament_size
        
        # Distance matrix oluştur
        self.distance_matrix = DistanceMatrixCache.get_matrix(cities, self.num_cities)
        
        # GA state
        self.population = []
        self.generation = 0
        self.best_distances = []
        self.avg_distances = []

    def initialize_population(self): # POP_SIZE KADAR RASTGELE ROTA OLUŞTURUR
        for _ in range(self.pop_size):
            tour = list(range(self.num_cities))  
            random.shuffle(tour)   
            
            individual = Individual(tour)        
            individual.calculate_fitness(self.distance_matrix) # HER ROTANIN FITNESS'I HESAPLANIR
            self.population.append(individual)

        self.population.sort(key=lambda x: x.get_total_distance()) # EN İYİ ROTADAN EN KÖTÜYE SIRALANIR
        
        best_distance = self.population[0].get_total_distance()
        worst_distance = self.population[-1].get_total_distance()
        avg_distance = sum(ind.get_total_distance() for ind in self.population) / len(self.population)
        
        print(f"✅ İlk populasyon hazır:")
        print(f"   En iyi: {best_distance:.2f}")
        print(f"   En kötü: {worst_distance:.2f}")
        print(f"   Ortalama: {avg_distance:.2f}")
        
        return best_distance

    def tournament_selection(self): # TOURNAMENT_SIZE KADAR RASTGELE BİREY ARASINDAN EN İYİYİ SEÇER
        tournament_size = min(self.tournament_size, len(self.population))
        tournament = random.sample(self.population, tournament_size)
        return min(tournament, key=lambda x: x.get_total_distance())
    
    def crossover(self, parent1, parent2): # İKİ EBEVEYNDEN BİR ÇOCUK
        size = len(parent1.tour_order)
        
        # rastgele 2 nokta seçilir
        start, end = sorted(random.sample(range(size), 2)) 
        child_tour = [-1] * size # bu satır ne yapıyor?
        child_tour[start:end] = parent1.tour_order[start:end]
        
        pointer = end
        for city in parent2.tour_order[end:] + parent2.tour_order[:end]:
            if city not in child_tour: # AVOID DUPLICATION
                child_tour[pointer % size] = city
                pointer += 1

        child = Individual(child_tour)
        child.calculate_fitness(self.distance_matrix)
        
        return child
    
    def mutate(self, individual): # İKİ RASTGELE ŞEHRİN YERİ DEĞİŞİR
        tour = individual.tour_order.copy()
        
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
        
        mutated = Individual(tour)
        mutated.calculate_fitness(self.distance_matrix)
        return mutated
        
    def evolve_generation(self):
        new_population = []
        mutation_count = 0

        new_population.extend(self.population[:self.elitism_count]) # en iyiler doğru bir şekilde aktarılıyor mu?

        while len(new_population) < self.pop_size:
             parent1 = self.tournament_selection()
             parent2 = self.tournament_selection()
    
             if random.random() < self.crossover_rate:
                 child = self.crossover(parent1, parent2)
             else:
                 # Crossover yapılmazsa parent1'i kopyala - MATRIX'İ BAĞLA!
                child = Individual(parent1.tour_order.copy())
                child.calculate_fitness(self.distance_matrix)
             
             if random.random() < self.mutation_rate:
                child = self.mutate(child)
                mutation_count += 1  # Mutation sayacı
            
             new_population.append(child)
        
        # Mutation istatistiği (debug için)
        if self.generation % 100 == 0:
            print(f"   Nesil {self.generation}: {mutation_count} mutasyon yapıldı")
        
        # Yeni populasyonu ata ve sırala
        self.population = new_population
        self.population.sort(key=lambda x: x.get_total_distance())
        
        # İstatistikleri güncelle
        distances = [ind.get_total_distance() for ind in self.population]
        self.best_distances.append(min(distances))
        self.avg_distances.append(sum(distances) / len(distances))
        
        self.generation += 1
        
    def run(self):
        print(f"🧬 GA başlıyor ({self.termination} nesil)...")
        
        t0 = time.perf_counter()
        initial = self.initialize_population()
        
        for gen in range(self.termination):
            self.evolve_generation()
            
            if gen % 100 == 0:
                current_best = self.population[0].get_total_distance()
                print(f"   Nesil {gen}: En iyi = {current_best:.2f}")
                
        t1 = time.perf_counter()
        
        # Final results
        best_solution = self.population[0]
        final_distance = best_solution.get_total_distance()
        improvement = ((initial - final_distance) / initial) * 100
        runtime = t1 - t0
        
        print(f"\n SONUÇLAR:")
        print(f"   İlk en iyi : {initial:.2f}")
        print(f"   Final en iyi : {final_distance:.2f}")
        print(f"   İyileştirme : {improvement:.1f}%")
        print(f"   En iyi rota : {best_solution.tour_order}")
        print(f"   Runtime : {runtime:.4f} s")
        
        return best_solution, runtime

    def plot_convergence(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.best_distances, label='En İyi', color='red', linewidth=2)
        plt.plot(self.avg_distances, label='Ortalama', color='blue', alpha=0.7)
        plt.xlabel('Nesil')
        plt.ylabel('Mesafe')
        plt.title(f'{self.num_cities} Şehir - GA Convergence')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    def plot_tour(self, individual=None):
        if individual is None:
            individual = self.population[0]
        
        plt.figure(figsize=(10, 8))
        
        # Cities
        x_coords = [city.x for city in self.cities]
        y_coords = [city.y for city in self.cities]
        plt.scatter(x_coords, y_coords, c='red', s=100, zorder=5)
        
        # City numbers
        for i, city in enumerate(self.cities):
            plt.annotate(str(i), (city.x, city.y), xytext=(5, 5), 
                        textcoords='offset points', fontsize=12)
        
        # Route
        route_x = [self.cities[city_idx].x for city_idx in individual.tour_order]
        route_y = [self.cities[city_idx].y for city_idx in individual.tour_order]
        route_x.append(route_x[0])  # Return to start
        route_y.append(route_y[0])
        
        plt.plot(route_x, route_y, 'b-', alpha=0.7, linewidth=2)
        plt.title(f'{self.num_cities} Şehir - En İyi Rota (Mesafe: {individual.get_total_distance():.2f})')
        plt.xlabel('X Koordinatı')
        plt.ylabel('Y Koordinatı')
        plt.grid(True, alpha=0.3)
        plt.show()

- **MATRIX CACHE :** 2 şehir arası uzaklığı her seferinde hesaplamak yerine rotadaki tüm ikili eşleşmeler tek seferde hesaplanıp matriste tutulur.

In [ ]:
class DistanceMatrixCache:
    _cache = {}  # Class-level cache
                 # Not be needed to create an object (instance)
    
    @classmethod
    def get_matrix(cls, cities, num): # matris varsa onu alır yoksa oluşturur
        if num in cls._cache: # cache'te var
            return cls._cache[num]
        
        matrix = cls.create_distance_matrix(cities) # cache'te yok oluştur
        cls._cache[num] = matrix
        return matrix
    
    @classmethod
    def create_distance_matrix(cls, cities):
        temp_individual = Individual()
        num_cities = len(cities)
        
        matrix = [[temp_individual.calculate_distance(cities[i], cities[j]) 
                  for j in range(num_cities)] 
                  for i in range(num_cities)]
        
        return matrix
    
    @classmethod
    def clear_cache(cls):
        cls._cache.clear()
    
    @classmethod
    def cache_info(cls):
        for i, num_cities in enumerate(cls._cache.keys()):
            print(f"   {i+1}. {num_cities} şehirli matrix")

### Dosyadan veri okuma

In [ ]:
def read_files():
    cities_dict = {}
    data_path = Path("Data")

    if not data_path.exists():
        print("Data klasörü bulunamadı!")
        return cities_dict
    
    for path in data_path.iterdir():
        if path.is_file() and path.name.startswith("tsp_"):
            print(path.name)
            try:
                with open(path, 'r') as file: # otomatik file.close() yapar

                    cities = []
                
                    try:
                        for line in file.readlines()[1:]: #2. satırdan itibaren okumaya başla
                            city_info = line.split()
                            city = City(float(city_info[0]), float(city_info[1]))
                            cities.append(city)
                    except ValueError as e:
                        print("Satır okunamadı.")
                    
                    cities_dict[len(cities)] = cities
                

            except Exception as e:
                print(f"{path.name} okunamadı")
    return cities_dict

In [ ]:
cities = read_files()
print(cities)

### ARRANGE THE CITY-LIST SIZES FOR TSP

In [ ]:
min_size = 0
avg_size = 0
max_size = 0

if 'cities' in globals() and cities:
    available_sizes = sorted(cities.keys())
    print(f"Available city list sizes: {available_sizes}")

    min_size = min(available_sizes) #   5 şehirli liste
    avg_size = available_sizes[1]   #  70 şehirli liste
    max_size = max(available_sizes) # 100 şehirli liste
    
    print(f"There are {min_size}, {avg_size}, {max_size} cities problems")

else:
    print("cities dict bulunamadı!")
    print("Önce şunu çalıştır: cities = read_files()")

### Try GA - Method

In [39]:
def try_ga(cities):
    try:
        ga = GA(cities=cities, pop_size=50, termination=200)
        
        best_solution, best_runtime = ga.run()
        
        print(f"En iyi mesafe: {best_solution.get_total_distance():.2f}")

        print("📉 Yakınsama grafiği çiziliyor...")
        ga.plot_convergence()
               
        print("🗺️ Route grafiği çiziliyor...")
        ga.plot_tour()
        
    except Exception as e:
        print(f"❌ Gerçek veri hatası: {e}")
        traceback.print_exc()

### Start GA - Method

In [ ]:
def start_ga(cities, param_grid):
    try:
        best_score = float('inf')
        best_runtime = float('inf')
        best_params = None
        best_individual = None
        best_ga = None

        for params in ParameterGrid(param_grid):
            print("Test ediliyor:", params)
            ga = GA(cities=cities, **params)
            best_individual, runtime = ga.run()
            best_distance = best_individual.get_total_distance()

            if best_distance < best_score or (best_distance == best_score and runtime < best_runtime):
                best_score = best_distance
                best_runtime = runtime
                best_params = params.copy()
                best_individual = best_individual
                best_ga = ga

        print(f"✅ {len(cities)} şehirli en iyi skor: {best_score}")
        print(f"🎯 {len(cities)} şehirli en iyi parametre: {best_params}")
        print(f"🧭 {len(cities)} şehirli en iyi runtime: {best_runtime}")
        print(f"🗺️ {len(cities)} şehirli en iyi rota grafiği:")
        best_ga.plot_tour(best_individual)

        print(f"📉 {len(cities)} şehirli yakınsama grafiği: ")
        best_ga.plot_convergence()

    except Exception as e:
        print(f"❌ Gerçek veri hatası: {e}")
        traceback.print_exc()

### 5-Cities TSP

In [ ]:
try_ga(cities[min_size])

### GridSearch for 5-cities TSP

In [ ]:
param_grid = {
    'mutation_rate': [0.01, 0.02],
    'termination': [50, 100, 200], 
    'pop_size': [50, 100, 200],
    'crossover_rate': [0.1, 0.5, 0.8],
    'elitism_rate': [0.05, 0.1],
    'tournament_size': [3, 5]
}

start_ga(cities[min_size], param_grid)

For 5-cities TSP, best parameters;
- crossover_rate = 0.1
- elitism_rate = 0.1
- mutation_rate = 0.01
- pop_size = 50,
- termination = 100, 
- tournament_size = 3

### 70-Cities TSP

In [ ]:
try_ga(cities[avg_size])

### GridSearch for 70-cities TSP

In [ ]:
param_grid = {
    'mutation_rate': [0.01, 0.02, 0.05],
    'termination': [200, 500, 1000], 
    'pop_size': [100, 1000, 5000],
    'crossover_rate': [0.1, 0.5, 0.8],
    'elitism_rate': [0.05, 0.1],
    'tournament_size': [3, 5]
}

start_ga(cities[avg_size], param_grid)

For 70-cities TSP, best parameters;
- mutation_rate = 0.05
- termination = 500, 
- pop_size = 5000, 
- crossover_rate = 0.8
- elitism_rate = 0.1
- tournament_size = 3

### 100-Cities TSP

In [ ]:
try_ga(cities[max_size])

### GridSearch for 100-cities TSP

In [ ]:
param_grid = {
    'mutation_rate': [0.01, 0.02, 0.05],
    'termination': [200, 500, 1000], 
    'pop_size': [100, 1000, 5000],
    'crossover_rate': [0.1, 0.5, 0.8],
    'elitism_rate': [0.05, 0.1],
    'tournament_size': [3, 5]
}

start_ga(cities[max_size], param_grid)

For 100-cities TSP, best parameters;
- mutation_rate = 0.02
- termination = 500, 
- pop_size = 5000, 
- crossover_rate = 0.8
- elitism_rate = 0.1
- tournament_size = 3